In [747]:
# Importing packages
import numpy as np
import pandas as pd
from sklearn import datasets, linear_model, preprocessing
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import copy
import time

## Package Linear Regression

In [750]:
# Importing the data
trainData0 = pd.read_csv('train.csv')
testData0 = pd.read_csv('test.csv')
trainData0.head() # viewing first few lines
testData0.head() # viewing first few lines

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1001,3131200640,20150427T000000,700000.0,4,2.00,1830,4590,2.0,0,...,8,1830,0,1908,0,98105,47.6593,-122.327,1650,4590
1,1002,984000710,20141022T000000,270000.0,3,2.00,1560,8853,1.0,0,...,7,1560,0,1967,0,98058,47.4312,-122.171,1610,8750
2,1003,4167300350,20140508T000000,258000.0,4,1.75,1730,8320,1.0,0,...,7,1230,500,1977,0,98023,47.3270,-122.361,1840,9800
3,1004,2826049282,20140614T000000,530000.0,3,2.50,1930,7214,2.0,0,...,8,1930,0,2005,0,98125,47.7191,-122.309,1930,7266
4,1005,8946750030,20141218T000000,245000.0,3,2.25,1422,3677,2.0,0,...,7,1422,0,2012,0,98092,47.3204,-122.178,1677,3677


In [58]:
# Removing unwanted features before training the data
trainDataFeatures = trainData0.drop(['price','Unnamed: 0','zipcode','lat','long'], axis=1)
testDataFeatures = testData0.drop(['price','id','date','Unnamed: 0','zipcode','lat','long'], axis=1)
trainDataFeatures.head() # viewing the results
testDataFeatures.head()

# Create 'target' dataframe to hold training labels (do for training and test)
trainDataTarget = trainData0['price']
testDataTarget = testData0['price']
# trainDataTarget.head() # viewing the results
# testDataTarget.head()


   bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
0         4       2.00         1830      4590     2.0           0     0   
1         3       2.00         1560      8853     1.0           0     0   
2         4       1.75         1730      8320     1.0           0     0   
3         3       2.50         1930      7214     2.0           0     0   
4         3       2.25         1422      3677     2.0           0     0   

   condition  grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
0          3      8        1830              0      1908             0   
1          3      7        1560              0      1967             0   
2          3      7        1230            500      1977             0   
3          3      8        1930              0      2005             0   
4          3      7        1422              0      2012             0   

   sqft_living15  sqft_lot15  
0           1650        4590  
1           1610        8750  
2          

In [751]:
# Next, training the model
# Reference: https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html
regr = linear_model.LinearRegression()
regr.fit(trainDataFeatures,trainDataTarget)
regr.coef_ # Outputting the coefficients


array([-2.46493614e+04,  2.37592759e+04,  9.02059584e+01,  5.50587659e-01,
        4.79461990e+04,  6.87629438e+05,  5.50927660e+04,  2.37133120e+03,
        9.56165482e+04,  2.51288274e+01,  6.50771311e+01, -3.47343747e+03,
        3.16902440e+01,  7.97486075e+01, -6.39773538e-01])

In [752]:
# Now, evaluating the model
trainPreds = regr.predict(trainDataFeatures)
trainMSE = mean_squared_error(trainDataTarget,trainPreds)
print(trainMSE)

36696274115.0656


In [753]:
# As sanity check, compare MSE to results using Stats Model
# Add a constant to the model for StatsModel regression
trainDataFeatures2 =sm.add_constant(trainDataFeatures)
regr_statsModel = sm.OLS(trainDataTarget,trainDataFeatures2).fit()
print(regr_statsModel.params) # Get the model coefficients
regr_statsModel.summary() # model summary

trainPreds_statsModel = regr_statsModel.predict(trainDataFeatures2)
print('\n\nMean Squared Error:')
print(mean_squared_error(trainDataTarget,trainPreds_statsModel)) # Get the MSE


const            6.161232e+06
bedrooms        -2.464936e+04
bathrooms        2.375928e+04
sqft_living      9.020595e+01
sqft_lot         5.505877e-01
floors           4.794620e+04
waterfront       6.876294e+05
view             5.509277e+04
condition        2.371331e+03
grade            9.561655e+04
sqft_above       2.512884e+01
sqft_basement    6.507714e+01
yr_built        -3.473437e+03
yr_renovated     3.169024e+01
sqft_living15    7.974861e+01
sqft_lot15      -6.397735e-01
dtype: float64


Mean Squared Error:
36696274115.0656


In [754]:
# Function to standardize the data frame
def standardize(df):
#     Function standardizes a dataframe so that all columns have mean 0 and standard deviation 1
#     Args: df is a data frame
#     Returns: normalized dataframe
    return((df-df.mean())/df.std())

In [755]:
# Standardize the training data
trainDataFeatures_standardized = standardize(trainDataFeatures)

# Check that mean is 0 and std is 1
trainDataFeatures_standardized.describe()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15
count,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03
mean,-1.376121e-16,6.945833e-17,1.437739e-16,3.608225e-17,4.150014e-16,6.731005e-16,1.035838e-16,9.969803e-17,3.789191e-16,-8.748557e-17,2.498002e-16,7.744916e-16,2.089995e-16,2.486344e-16,1.573741e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.930695e+00,-2.834928e+00,-1.882127e+00,-4.852412e-01,-8.630450e-01,-8.975774e-02,-3.097533e-01,-3.574473e+00,-3.108032e+00,-1.734429e+00,-6.672526e-01,-2.449339e+00,-2.066570e-01,-1.725849e+00,-5.115550e-01
25%,-4.096185e-01,-7.562810e-01,-7.249407e-01,-3.202091e-01,-8.630450e-01,-8.975774e-02,-3.097533e-01,-6.731150e-01,-5.223148e-01,-7.092127e-01,-6.672526e-01,-6.047702e-01,-2.066570e-01,-7.414198e-01,-3.224846e-01
50%,-4.096185e-01,-6.339873e-02,-1.702793e-01,-2.298635e-01,-8.630450e-01,-8.975774e-02,-3.097533e-01,-6.731150e-01,-5.223148e-01,-2.662182e-01,-6.672526e-01,1.756242e-01,-2.066570e-01,-2.044585e-01,-2.192521e-01
75%,7.640735e-01,6.294836e-01,4.744793e-01,-1.109365e-01,1.069867e+00,-8.975774e-02,-3.097533e-01,7.775639e-01,3.395908e-01,4.837083e-01,6.190688e-01,8.141287e-01,-2.066570e-01,5.562367e-01,-1.232922e-01
max,4.285150e+00,4.093895e+00,4.526041e+00,1.038195e+01,3.969234e+00,1.112996e+01,4.918150e+00,2.228243e+00,3.787213e+00,5.467397e+00,3.901406e+00,1.629996e+00,4.884624e+00,4.135979e+00,8.785990e+00


In [756]:
regr_standardized = linear_model.LinearRegression()
regr_standardized.fit(trainDataFeatures_standardized,trainDataTarget)

# Evaluate the model
trainPreds_standardized = regr_standardized.predict(trainDataFeatures_standardized)
trainMSE_standardized = mean_squared_error(trainDataTarget,trainPreds_standardized)

# Compare results to model trained on non-standardized features
print('Standardized MSE:')
print(trainMSE_standardized)

print('\n\nNon-standardized MSE:')
print(trainMSE)


Standardized MSE:
36696274115.0656


Non-standardized MSE:
36696274115.0656


In [757]:
# Perform linear regression on the testing data 

# First model trained on non-standardized data
testPreds = regr.predict(testDataFeatures) # get predictions
testMSE = mean_squared_error(testDataTarget,testPreds) # calculate MSE

# Next standardized Case
testDataFeatures_standardized = standardize(testDataFeatures) # standardize the test data
testPreds_standardized = regr_standardized.predict(testDataFeatures_standardized) # get predictions
testMSE_standardized = mean_squared_error(testDataTarget,testPreds_standardized) # calculate MSE

# Display results
# Compare results to model trained on non-standardized features
print('Standardized MSE:')
print(testMSE_standardized)

print('\n\nNon-standardized MSE:')
print(testMSE)


Standardized MSE:
64271736305.60536


Non-standardized MSE:
62548921543.78229


In [758]:
print(testDataFeatures_standardized.head())
print(trainDataFeatures_standardized.head())

   bedrooms  bathrooms  sqft_living  sqft_lot    floors  waterfront      view  \
0  0.619401  -0.127594    -0.284992 -0.195613  1.129167   -0.114709 -0.328948   
1 -0.452227  -0.127594    -0.572626 -0.122960 -0.848359   -0.114709 -0.328948   
2  0.619401  -0.442642    -0.391523 -0.132044 -0.848359   -0.114709 -0.328948   
3 -0.452227   0.502501    -0.178462 -0.150893  1.129167   -0.114709 -0.328948   
4 -0.452227   0.187453    -0.719638 -0.211173  1.129167   -0.114709 -0.328948   

   condition     grade  sqft_above  sqft_basement  yr_built  yr_renovated  \
0  -0.691127  0.314438    0.047055      -0.681428 -2.128144     -0.229295   
1  -0.691127 -0.515213   -0.267478      -0.681428 -0.019160     -0.229295   
2  -0.691127 -0.515213   -0.651908       0.425095  0.338295     -0.229295   
3  -0.691127  0.314438    0.163549      -0.681428  1.339169     -0.229295   
4  -0.691127 -0.515213   -0.428240      -0.681428  1.589388     -0.229295   

   sqft_living15  sqft_lot15  
0      -0.492580   

In [759]:
# Get statistics using Stats Model
# Add a constant to the model for StatsModel regression
testDataFeatures2 = sm.add_constant(testDataFeatures)
testPreds_statsModel = regr_statsModel.predict(testDataFeatures2)
mean_squared_error(testDataTarget,testPreds_statsModel)
regr_statsModel.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.681
Model:                            OLS   Adj. R-squared:                  0.677
Method:                 Least Squares   F-statistic:                     150.4
Date:                Sat, 09 Feb 2019   Prob (F-statistic):          5.57e-233
Time:                        12:48:56   Log-Likelihood:                -13582.
No. Observations:                1000   AIC:                         2.719e+04
Df Residuals:                     985   BIC:                         2.727e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          6.161e+06   5.92e+05     10.405      0.000       5e+06    7.32e+06
bedrooms      -2.465e+04   9051.974     -2.723      0.007   -4.24e+04   -6885.992
bathrooms      2.376e+04   1.46e+04      1.623      0.105   -4963.650    5.25e+04
sqft_living      90.2059     10.452      8.631      0.000      69.695     110.717
sqft_lot          0.5506      0.385      1.431      0.153      -0.204       1.306
floors         4.795e+04   1.64e+04      2.924      0.004    1.58e+04    8.01e+04
waterfront     6.876e+05   7.55e+04      9.108      0.000    5.39e+05    8.36e+05
view           5.509e+04   9636.305      5.717      0.000    3.62e+04     7.4e+04
condition      2371.3312   9725.294      0.244      0.807   -1.67e+04    2.15e+04
grade          9.562e+04   9590.284      9.970      0.000    7.68e+04    1.14e+05
sqft_above       25.1288     10.785      2.330      0.020       3.964      46.294
sqft_basement    65.0771     11.618      5.602      0.000      42.279      87.876
yr_built      -3473.4375    304.239    -11.417      0.000   -4070.469   -2876.406
yr_renovated     31.6902     16.805      1.886      0.060      -1.288      64.668
sqft_living15    79.7486     17.367      4.592      0.000      45.668     113.829
sqft_lot15       -0.6398      0.449     -1.425      0.154      -1.521       0.241
==============================================================================
Omnibus:                      361.731   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3638.012
Skew:                           1.361   Prob(JB):                         0.00
Kurtosis:                      11.939   Cond. No.                     2.52e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.75e-23. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [760]:
# Make StatsModel for standardized features (to evaluate model)
trainDataFeatures2_standardized = sm.add_constant(trainDataFeatures_standardized) # adding constant to training features

t1 = time.time()
regr_standardized_statsModel = sm.OLS(trainDataTarget,trainDataFeatures2_standardized).fit() # training model
t2 = time.time()
print('elapsed time:',t2-t1)
# print(regr_standardized_statsModel.summary())
# regr_standardized_statsModel.params

# Adding constant to test features
testDataFeatures2_standardized = sm.add_constant(testDataFeatures_standardized)

# Testing the model
testPreds_standardized_statsModel = regr_standardized_statsModel.predict(testDataFeatures2_standardized)
regr_standardized_statsModel_MSE = mean_squared_error(testDataTarget,testPreds_standardized_statsModel)

# Calculating error
error = testPreds_standardized_statsModel-testDataTarget
df = pd.DataFrame({'Actual':testDataTarget, 'Prediction':testPreds_standardized_statsModel,'Error':error})
print(df[:10]) # Printing Actual,Prediction,Error
print('\n\nMeans:')
print(np.mean(df)) # Getting mean for actual/prediction
print('\n\nMean Error:')
print(np.sqrt(regr_standardized_statsModel_MSE))





elapsed time: 0.002126932144165039
     Actual     Prediction          Error
0  700000.0  685022.720776  -14977.279224
1  270000.0  326531.052836   56531.052836
2  258000.0  320816.292712   62816.292712
3  530000.0  410325.389516 -119674.610484
4  245000.0  215246.357781  -29753.642219
5  563000.0  490201.009410  -72798.990590
6  515000.0  522670.709494    7670.709494
7  325000.0  668159.145163  343159.145163
8  540000.0  403382.282990 -136617.717010
9  280300.0  243337.405444  -36962.594556


Means:
Actual        541676.554
Prediction    520414.834
Error         -21261.720
dtype: float64


Mean Error:
253518.7099714839


## Linear Regression

In [761]:
# Simple linear regression for one feature
X0 = trainDataFeatures_standardized['sqft_lot'] # Get standardized training data for sqft_lot feature
X = np.matrix(sm.add_constant(X0)) # Add ones to the data frame, and convert to matrix
Y = np.matrix(trainDataTarget).transpose() # Response variable: convert to matrix

XtX_inv =  np.linalg.inv(np.matmul(X.transpose(),X)) # Get XtX_inv
XtY = np.matmul(X.transpose(),Y) # Get XtY
Theta = np.matmul(XtX_inv,XtY) # Multiply to obtain parameter values
print(Theta) # print out result


[[520414.834     ]
 [ 49784.22998124]]


In [762]:
# Get mean and std from training data for standardization
mean_sqft_lot = np.mean(trainDataFeatures['sqft_lot'])
std_sqft_lot = np.std(trainDataFeatures['sqft_lot'])

# Predict price based on single feature, sqft_lot
def predPrice_from_sqft_lot(x):
    # Function predicts price of a home based on 'sqft_lot' feature, using simple linear regression model
    # Args: x represents the 'sqft_lot' for a given home (standardized)
    # Returns: price representing the estimated price for the home
    X_standardized = (x-mean_sqft_lot)/std_sqft_lot # standardize the input relative to training set
    X1 = np.matrix([1,X_standardized]) # add constant for matrix multiplication
    return(np.matmul(X1,Theta)) # return results of model
    

In [763]:
# Multiple linear regression model
X0 = trainDataFeatures_standardized # Get standardized training data for sqft_lot feature
X = np.matrix(sm.add_constant(X0)) # Add ones to the data frame, and convert to matrix
Y = np.matrix(trainDataTarget).transpose() # Response variable: convert to matrix

XtX_inv =  np.linalg.inv(np.matmul(X.transpose(),X)) # Get XtX_inv
XtY = np.matmul(X.transpose(),Y) # Get XtY
Theta = np.matmul(XtX_inv,XtY) # Multiply to obtain parameter values
print(Theta) # print out coefficients

[[520414.834     ]
 [-24856.49629834]
 [ 46975.33942369]
 [ 19946.06997127]
 [ 13621.15073323]
 [ 17075.56460921]
 [ 61063.34582439]
 [ 43969.84516534]
 [  2060.36103052]
 [105829.15371893]
 [ 69065.61855591]
 [ 45228.44176929]
 [-97919.23491527]
 [ 12535.97127341]
 [ 53466.60481998]
 [-16054.36807954]]


In [764]:
# Predict price based on all features
def predPrice(X):
    # Function predicts price of a home using multiple linear regression model
    # Args: X is a data frame representing the standardized observations (i.e. test data)
    # Returns: Vector representing the estimated price for each observation
    X1 = sm.add_constant(X) # add constant to input data
    X1_mat = np.matrix(X1) # convert to matrix
    return(np.matmul(X1_mat,Theta)) # return results of model

In [765]:
# Evaluate model on training set
trainPreds = predPrice(trainDataFeatures_standardized) # Get predictions
print('MSE on training set (multiple linear regression):')
print(mean_squared_error(trainDataTarget,trainPreds)) # Calculate MSE

# Evaluate model on test set
testPreds = predPrice(testDataFeatures_standardized) # Get predictions
print('\n\nMSE on test set (multiple linear regression):')
print(mean_squared_error(testDataTarget,testPreds)) # Calculate MSE

MSE on training set (multiple linear regression):
37108346146.44747


MSE on test set (multiple linear regression):
63825578226.78736


## Gradient Descent

In [766]:
# Gradient descent algorithm
def GradDescent(X,Y,alpha,epsilon,maxIter):
    # Gradient descent algorithm for linear regression
    # Args: X is an (n x d) data frame of training features (with each row representing an instance)
    #       Y is a (n x 1) data frame of training target data
    # Returns: Theta is an (d x 1) array of optimized parameters for linear regression
    X1 = np.matrix(sm.add_constant(X)) # add constant to input data
    Y = np.matrix(Y).transpose() # convert response variable to (n x 1) matrix
    d = X1.shape[1] # number of features
    n = X1.shape[0] # number of training examples
    theta = np.ones((d,1)) # Initialize theta with ones
    delta_theta = epsilon+1 # initialize delta_theta
    itr = 1 # initialize iteration

    # Begin iterating
    while (itr < maxIter+1) & (delta_theta > epsilon):
        loss = np.matmul(X1,theta)-Y # Calculate loss (objective)
        loss = np.squeeze(np.asarray(loss)) # convert to array
        idx = 0 # index used in "for loop"
        theta0 = copy.deepcopy(theta) # record value of theta for comparison to new value
        for idx in range(0,d):
            theta[idx] = theta[idx] - (alpha * (2/n) * np.sum(loss * X1[:,idx])) # update each feature's parameter
            idx = idx + 1 # move to next feature
        delta_theta = np.sqrt(sum((theta-theta0)**2)) # Get change in theta 
#         if (itr % 25 == 0): # Track progress if desired
#             print('Iteration',itr)
#             print('Delta Theta',delta_theta)
        itr = itr+1
    return(theta)


In [767]:
# Predict price based on all features
def predPrice2(X,theta):
    # Function predicts price of a home using multiple linear regression model
    # Args: X is a data frame representing the standardized observations (i.e. test data)
    # Returns: Vector representing the estimated price for each observation
    X1 = sm.add_constant(X) # add constant to input data
    X1_mat = np.matrix(X1) # convert to matrix
    return(np.matmul(X1_mat,theta)) # return results of model

In [768]:
alpha = .1 # learning rate for gradient descent
epsilon = .01 # threshold for stopping criteria
maxIter = 1000 # maximum number of iterations

# Perform gradient descent
t1 = time.time()
theta = GradDescent(trainDataFeatures_standardized,trainDataTarget,alpha,epsilon,maxIter)
t2 = time.time()
print('elapsed time:',t2-t1) # Time the algorithm

# Get predictions from learned model
trainPreds = predPrice2(trainDataFeatures_standardized,theta)
testPreds = predPrice2(testDataFeatures_standardized,theta)

# Calculate MSE for both
trainMSE = mean_squared_error(trainDataTarget,trainPreds)
testMSE = mean_squared_error(testDataTarget,testPreds)

# Print results
print(trainMSE)
print(testMSE)
print(theta)

elapsed time: 0.17957282066345215
36696274115.0802
64271739582.59096
[[520414.834     ]
 [-21001.54650853]
 [ 17145.26376992]
 [ 56906.49093436]
 [ 15945.38682001]
 [ 24805.18158307]
 [ 61287.57786672]
 [ 42152.84671856]
 [  1634.62672697]
 [110936.21458474]
 [ 40488.05707715]
 [ 41119.23153732]
 [-97919.25372088]
 [ 12535.97126078]
 [ 53466.68169664]
 [-16054.16445617]]


## Ridge Regression

In [770]:
# Ridge Regression
X0 = trainDataFeatures_standardized # Get standardized training data
X = np.matrix(sm.add_constant(X0)) # Add ones to the data frame, and convert to matrix
Y = np.matrix(trainDataTarget).transpose() # Response variable: convert to matrix
d = X.shape[1] # Number of features
lam = 10; # Value for lambda (regularization parameter)
lamMat = lam*np.identity(d) # Multiply by identity matrix

XtX_plus_lamMat_inv =  np.linalg.inv(np.matmul(X.transpose(),X)+lamMat) # Get XtX_inv
XtY = np.matmul(X.transpose(),Y) # Get XtY
Theta = np.matmul(XtX_plus_lamMat_inv,XtY) # Multiply to obtain parameter values
print(Theta) # print out result

[[515262.21188119]
 [-20364.58383067]
 [ 17023.06223465]
 [ 57124.10676888]
 [ 14910.55412327]
 [ 24376.03221788]
 [ 60682.3179826 ]
 [ 42438.41465786]
 [  2106.88013165]
 [108393.60093607]
 [ 40828.94444296]
 [ 40949.67437777]
 [-95708.79106837]
 [ 13044.0485009 ]
 [ 53639.72014662]
 [-15107.18118963]]


In [773]:
# Evaluate the model
# First get predictions
trainPreds = predPrice2(trainDataFeatures_standardized,Theta)
testPreds = predPrice2(testDataFeatures_standardized,Theta)

# Calculate MSE
trainMSE = mean_squared_error(trainDataTarget,trainPreds) # Calculate MSE
testMSE = mean_squared_error(testDataTarget,testPreds) # Calculate MSE

# Print results
print(trainMSE)
print(testMSE)

36728130809.96714
64615803001.5835
